In [0]:
%python


data = [(101, 'Alice','20223-11-01', 2000), ('102','Bob', '2023-11-02', 3000)]

cols = ['order_id','customer','order_date','amount']

df=spark.createDataFrame(data,cols)
display(df)

In [0]:
%python
df.write.format("delta").mode("overwrite").save("dbfs:/user/hive/warehouse/bronze_sales")

In [0]:
df.write.format("delta").mode("overwrite").option("ovewriteSchema","true").save("/FileStore/bronze_sales")

In [0]:
df.write.format("delta").mode("overwrite").save("/tmp/lakehouse/bronze/customers")

In [0]:
df.write.format("delta").mode("overwrite").saveAsTable("bronze_customers")

In [0]:
from pyspark.sql.functions import col, lower , trim

df_bronze=spark.read.table("bronze_customers")

df_bronze.printSchema()

In [0]:
from pyspark.sql.functions import col, lower , trim
df_silver= df_bronze.withColumn('customer', lower(trim(col("customer_name")))).dropna(subset=["order_id","customer"])


df_silver.write.format('delta').mode('overwrite').saveAsTable('silver_customers')

display(df_silver)


In [0]:
df_silver = spark.read.table("silver_customers")

df_gold = df_silver.groupBy("state").count